In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
from tifffile import imread 
from skimage.measure import regionprops, label 
import os
import csv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# function to measure distance
def get_distance(centroid1, centroid2):
    x_distance = (centroid1[0] - centroid2[0])**2
    y_distance = (centroid1[1] - centroid2[1])**2
    distance = np.sqrt(x_distance+y_distance)
    return distance

def get_intersection(coords_1, coords_2):
    # function to find coordinates that are present in both matched objects
    intersection = 0
    
    for c1 in range(len(coords_1)):
        for c2 in range(len(coords_2)):
            if coords_1[c1][0] ==coords_2[c2][0] and coords_1[c1][1] ==coords_2[c2][1]:
                intersection = intersection + 1
    return intersection

def get_union(coords_1, coords_2, intersection):
    # function to find coordinates that are present in one, other, or both
    # matched objects (but don't want to count twice!)
    union = len(coords_1) + len(coords_2) - intersection
    return union

def get_jaccard(coords_1, coords_2):
    # jaccard index is intersection divided by union
    intersection = get_intersection(coords_1, coords_2)
    union = get_union(coords_1, coords_2, intersection)
    return intersection/union


In [13]:

# loops through the different folders of images
for i in range(0, 5):
    all_jaccard = []

    file_num = "00"+str(i)

    # setting path to directory containing annoated images of the of images 
    parent_folder = "/Volumes/Untitled/240116/AV1200/Annotated_Files/"+file_num
    
    # retriving the file and ensuring it is the correct file
    if os.path.isdir(parent_folder):
        all_files = os.listdir(parent_folder)
    
    # accessing it through an external drive, so ensuring only the necessary (.tif) files are taken
    tif_files = []
    for file in all_files:
        if file.endswith(".tif") and file.startswith(file_num):
            tif_files.append(file)
    test_objects = {}
    
    # seperating the .tif files into the ground truth and the test objects
    for file in tif_files:
        if "500ms" in file:
            gt_path = os.path.join(parent_folder, file)
        else:
            test_objects[file] = os.path.join(parent_folder, file)

    # reads, labels, and retrieves the regionprop for the ground truth image and the current test object image
    for t_object in test_objects:
        gt_image = imread(gt_path)
        test_image = imread(test_objects[t_object]) 
        
        label_gt =  label(gt_image) 
        label_test =  label(test_image) 
    
        # measure each object in image
        props_gt = regionprops(label_gt)
        props_test = regionprops(label_test)
    
        map= {} # GT_object: test_object
    
        for GT_object in range(len(props_gt)):
            for test_object in range(len(props_test)):
                # get distance between GT centroid to test centroid
                distance_found = get_distance(props_gt[GT_object].centroid, props_test[test_object].centroid)
                if test_object ==0:
                    min_distance = distance_found
                    distance_index = test_object
                else:
                    if min_distance > distance_found:
                        min_distance = distance_found
                        distance_index = test_object
            map[GT_object] = distance_index
    
        # retrieves jaccard index and saves in to array
        jaccard_index = []
        for GTobject in range(len(map)):
            coords_1 = props_gt[GTobject].coords
            coords_2 = props_test[map[GTobject]].coords
        
            jaccard = get_jaccard(coords_1, coords_2)
            if jaccard == 0:
                jaccard = np.nan
            jaccard_index.append(jaccard)

        
        # creates CSV files for the jaccard index
        filename = 'Jaccard Index ' + file_num+ "_"+t_object[15:-4]+'.csv'
        filepath = os.path.join(parent_folder, filename)
        file = open(filepath, mode='w', newline='')
        csvwriter = csv.writer(file)
        csvwriter.writerow(["Nucleus", "Test_Object", "Jaccard_Index"])
    
        for GT_object in map:
            test_object = map[GT_object]
            csvwriter.writerow([GT_object, test_object, jaccard_index[GT_object]])  
        file.close()

        # creates CSV files for the jaccard index
        filename = 'Nucleus Object Attributes ' + file_num+ "_"+t_object[15:-4]+'.csv'
        filepath = os.path.join(parent_folder, filename)
        file = open(filepath, mode='w', newline='')
        csvwriter = csv.writer(file)
        csvwriter.writerow(["GT_Object", "Nucleus", "Area", "Circularity", "Eccentricity", "Major Axis Length", "Minor Axis Length", "Orientation"])

        count = 0
        for GT_object in map:
            test_object = map[GT_object]
            area = props_test[test_object].area
            perimeter = props_test[test_object].perimeter
            circularity = (4 * np.pi * area) / (perimeter ** 2)
            eccentricity = props_test[test_object].eccentricity
            major_axis = props_test[test_object].axis_major_length
            minor_axis = props_test[test_object].axis_minor_length
            orientation = props_test[test_object].orientation
            csvwriter.writerow([GT_object, count, area, circularity, eccentricity, major_axis, minor_axis, orientation ])
            count = count + 1  
        file.close()
        
    # creates CSV files for the ground truth image
    filename = 'Ground Truth Object Attributes ' + file_num +'.csv'
    filepath = os.path.join(parent_folder, filename)
    file = open(filepath, mode='w', newline='')
    csvwriter = csv.writer(file)
    csvwriter.writerow(["Nucleus", "Area", "Circularity", "Eccentricity", "Major Axis Length", "Minor Axis Length", "Orientation"])
    
    count = 0
    for GTprop_test in props_gt:
        area = GTprop_test.area
        perimeter = GTprop_test.perimeter
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        eccentricity = GTprop_test.eccentricity
        major_axis = GTprop_test.axis_major_length
        minor_axis = GTprop_test.axis_minor_length
        orientation = GTprop_test.orientation
        csvwriter.writerow([count, area, circularity, eccentricity, major_axis, minor_axis, orientation ])
        count = count + 1  
    file.close()
